<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_15/S15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Install packages

In [ ]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
#!cd ../
!git clone "https://github.com/jyanivaddi/dl_hub.git"
!git -C dl_hub pull
!git pull
#!cd ../

!pip install --quiet "torchinfo" "seaborn" "pytorch-lightning" "torchmetrics" "lightning-bolts", "torchtext", "datasets", "tokenizers"
!pip install --quiet "prettytable"
!pip install --quiet "torchsummary"
!pip install --quiet "torch_lr_finder"
!pip install --quiet "grad-cam"
!pip install --quiet "gradio"

In [ ]:
import sys
sys.path.append("ERA_V1/session_15")
sys.path.append("dl_hub")

In [1]:
!git -C dl_hub pull


remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 347 bytes | 347.00 KiB/s, done.
From https://github.com/jyanivaddi/dl_hub
   046aef8..d8a625d  main       -> origin/main
Updating 046aef8..d8a625d
Fast-forward
 Transformer/PL_main.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [2]:
import torch
import torch.nn as nn
#from dl_hub.Transformer.config import get_config
from config import get_config
from train import train_model
from PL_data_module import OpusDataModule
from PL_model import LitTransformer
from PL_main import train_transformer


In [3]:
cfg = get_config()
cfg['batch_size'] = 16
cfg['d_model'] = 512
cfg['preload'] = None
cfg['num_epochs'] = 10


**Define data module**

In [4]:
opus_dm = OpusDataModule(cfg)
opus_dm.prepare_data()
opus_dm.setup()

**Define PL Model**

In [5]:
tokenizer_src = opus_dm.tokenizer_src
tokenizer_tgt = opus_dm.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = LitTransformer(cfg, loss_criterion, tokenizer_src, tokenizer_tgt)
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)


/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'loss_criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_criterion'])`.
  rank_zero_warn(


In [ ]:
train_transformer(transformer_model, opus_dm, cfg, epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 75.1 M
----------------------------------------------------
75.1 M    Trainable params
0         Non-trainable params
75.1 M    Total params
300.532   Total estimated model params size (MB)
/home/e183534/ERA_V1/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/e183534/ERA_V1/venv

Training: 0it [00:00, ?it/s]

In [ ]:
!tensorboard --logdir=runs